In [1]:
import cv2
import mediapipe as mp
import D_Ham
import numpy as np
import os

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def make_pose(results, c_lm):
    c_lm.append(results.pose_landmarks.landmark[9].x)
    c_lm.append(results.pose_landmarks.landmark[9].y)

    c_lm.append(results.pose_landmarks.landmark[10].x)
    c_lm.append(results.pose_landmarks.landmark[10].y)
    
    return

In [6]:
def make_l_hand(results, c_lm):
    for id, lm in enumerate(results.left_hand_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
    return

def make_r_hand(results, c_lm):
    for id, lm in enumerate(results.right_hand_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
    return

# Data Camera

In [10]:
cap = cv2.VideoCapture(0)
lm_list = []
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    while True:
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        if results.pose_landmarks:
            if results.left_hand_landmarks or results.right_hand_landmarks:
                lm = []
                make_pose(results, lm)
                if results.left_hand_landmarks:
                    make_l_hand(results, lm)
                else:
                    for j in range(42):
                        lm.append(0)
                if results.right_hand_landmarks:
                    make_r_hand(results, lm)
                else:
                    for j in range(42):
                        lm.append(0)
                lm_f = D_Ham.FindPoint_H(frame, lm)
                lm_pre = D_Ham.pre_process_H(lm_f)
                if len(lm_list) < 16:
                    lm_list.append(lm_pre)
                print(len(lm_list))

        cv2.imshow("Image", image)
        key = cv2.waitKey(11)
        if key == 27:
            break
        elif key == 8:
            lm_list = []
        elif key == 49:
            save_path = './model/Point_History/Point/' + str(26) + '.csv'
            D_Ham.SavePoint_H(26, lm_list, save_path)
            lm_list = []
        
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16
16
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
16
16
16


In [8]:
cap.release()
cv2.destroyAllWindows()

# Data Video

In [9]:
import csv
def SavePoint_H(key, list, path):
    with open(path, 'a', newline="") as f:
        writer = csv.writer(f)
        for i in range(4):
            for val in list:
                writer.writerow([key, *val])
    return

In [10]:
def ReadLb_H(path):
    with open(path,
              encoding='utf-8-sig') as f:
        keypoint_labels = csv.reader(f)
        keypoint_labels = [
            row[0] for row in keypoint_labels
        ]
    return keypoint_labels

In [11]:
import json
f = open('D:/KLTN/videodata/videos/nslt_2000.json',)
data = json.load(f)
f.close()
tam = ReadLb_H('./model/Point_History/ID.csv')

In [12]:
for j in range(14):
    if int(tam[j]) == 312:
        print(j)

12


In [13]:
lm_list = []
for i in data:
    id = data[i]["action"][0]
    for j in range(14):
        if id == int(tam[j]):
            path = "D:/KLTN/videodata/dataset/" + str(j) + "/"  + i + ".mp4"

            if os.path.exists(path):
                cap = cv2.VideoCapture(path)
                with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.5) as holistic:
                    while True:
                        ret, frame = cap.read()
                        if not ret:
                            break
                        image, results = mediapipe_detection(frame, holistic)
                        draw_landmarks(image, results)
                        if results.pose_landmarks:
                            if results.left_hand_landmarks or results.right_hand_landmarks:
                                lm = []
                                make_pose(results, lm)
                                if results.left_hand_landmarks:
                                    make_l_hand(results, lm)
                                else:
                                    for j in range(42):
                                        lm.append(0)
                                if results.right_hand_landmarks:
                                    make_r_hand(results, lm)
                                else:
                                    for j in range(42):
                                        lm.append(0)
                                lm_f = D_Ham.FindPoint_H(frame, lm)
                                lm_pre = D_Ham.pre_process_H(lm_f)
                                lm_list.append(lm_pre)
                        cv2.imshow('OpenCV Feed', image)
                        if cv2.waitKey(10) & 0xFF == ord('q'):
                            break
            
                lm_save = []
                if len(lm_list) != 16:
                    lm_save.append(lm_list[0])
                    for s in range(14):
                        lm_save.append(lm_list[int((s + 1) * len(lm_list)/16)])
                    lm_save.append(lm_list[len(lm_list) - 1])
                else:
                    lm_save = lm_list
                save_path = './model/Point_History/Point/' + str(j) + '.csv'
                D_Ham.SavePoint_H(j, lm_save, save_path)

                lm_list = []

cap.release()
cv2.destroyAllWindows()
    

In [ ]:
lm_list = []
for i in data:
    id = data[i]["action"][0]
    if  id == 402:
        path = "D:/KLTN/videodata/videos/" + str(id) + "/"  + i + ".mp4"

        if os.path.exists(path):
            cap = cv2.VideoCapture(path)
            with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.5) as holistic:
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    image, results = mediapipe_detection(frame, holistic)
                    draw_landmarks(image, results)
                    if results.pose_landmarks:
                        if results.left_hand_landmarks or results.right_hand_landmarks:
                            lm = []
                            make_pose(results, lm)
                            if results.left_hand_landmarks:
                                make_l_hand(results, lm)
                            else:
                                for j in range(42):
                                    lm.append(0)
                            if results.right_hand_landmarks:
                                make_r_hand(results, lm)
                            else:
                                for j in range(42):
                                    lm.append(0)
                            lm_f = D_Ham.FindPoint_H(frame, lm)
                            lm_pre = D_Ham.pre_process_H(lm_f)
                            lm_list.append(lm_pre)
                    cv2.imshow('OpenCV Feed', image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
          
            lm_save = []
            if len(lm_list) != 16:
                lm_save.append(lm_list[0])
                for s in range(14):
                    lm_save.append(lm_list[int((s + 1) * len(lm_list)/16)])
                lm_save.append(lm_list[len(lm_list) - 1])
            else:
                lm_save = lm_list
            save_path = './model/Point_History/Point/' + str(0) + '.csv'
            SavePoint_H(0, lm_save, save_path)

            lm_list = []

    # lm_save = []
    # if len(lm_list) > 16:
    #     lm_save.append(lm_list[0])
    #     for s in range(14):
    #         lm_save.append(lm_list[int((s + 1) * len(lm_list)/16)])
    #     lm_save.append(lm_list[len(lm_list) - 1])

    # save_path = './model/Point_History/ToaDo/' + str(id) + '.csv'
    # ham.SavePoint_H(id, lm_save, save_path)

    # lm_list = []
      
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()